In [21]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from credentials import mysql_username, mysql_password

# create the SparkSession
spark = SparkSession.builder.appName('capstone-pyspark').getOrCreate()


In [3]:
# Load / Read data from json file:
df_branch = spark.read.json("cdw_sapp_branch.json")
df_branch.show()

+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|       Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|  Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|      Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL|   Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|        14th Street|     19406|2018-04-18T16:51:...|
|         Paters

In [4]:
df_branch.columns
df_branch.printSchema()
df_branch.describe().show()
df_branch.show()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)

+-------+-----------+-----------------+------------+--------------------+------------+-------------+------------------+--------------------+
|summary|BRANCH_CITY|      BRANCH_CODE| BRANCH_NAME|        BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|        BRANCH_ZIP|        LAST_UPDATED|
+-------+-----------+-----------------+------------+--------------------+------------+-------------+------------------+--------------------+
|  count|        115|              115|         115|                 115|         115|          115|               115|                 115|
|   mean|       null|76.67826086956522|        null|1.2345499259478261E9|        null|

In [5]:
#df_branch.select('BRANCH_CITY', 'BRANCH_STATE','BRANCH_ZIP')\
    #.where(F.length(df_branch["BRANCH_ZIP"]) < 5).show()

In [6]:
df_branch.filter(F.length(df_branch["BRANCH_ZIP"]) < 5)\
         .select('BRANCH_CITY', 'BRANCH_STATE', 'BRANCH_ZIP')\
         .show()


+------------+------------+----------+
| BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP|
+------------+------------+----------+
|    Paterson|          NJ|      7501|
|Wethersfield|          CT|      6109|
|Hillsborough|          NJ|      8844|
|     Medford|          MA|      2155|
|    Rockaway|          NJ|      7866|
|  LongBranch|          NJ|      7740|
|   Irvington|          NJ|      7111|
|    NewHaven|          CT|      6511|
|      Quincy|          MA|      2169|
+------------+------------+----------+



In [7]:
df_branch = df_branch.withColumn('BRANCH_ZIP',\
                    F.when((F.length(df_branch['BRANCH_ZIP']) == 4) &
                        df_branch['BRANCH_STATE'].isin(["NJ", "CT", "MA"]),
                    F.format_string("0%s",df_branch['BRANCH_ZIP']))\
                    .otherwise(df_branch["BRANCH_ZIP"]))


In [8]:
df_branch.select('BRANCH_ZIP')\
    .where(F.length(df_branch["BRANCH_ZIP"]) < 4).show()

+----------+
|BRANCH_ZIP|
+----------+
+----------+



In [9]:
'''df_branch = df_branch.withColumn(
    'BRANCH_ZIP',
    F.when(
        (F.length(df_branch['BRANCH_ZIP']) == 4) &
        df_branch['BRANCH_STATE'].isin(["NJ", "CT", "MA"]),
        F.concat(F.lit("0"), df_branch['BRANCH_ZIP'])
    ).otherwise(df_branch["BRANCH_ZIP"])
)'''

'df_branch = df_branch.withColumn(\n    \'BRANCH_ZIP\',\n    F.when(\n        (F.length(df_branch[\'BRANCH_ZIP\']) == 4) &\n        df_branch[\'BRANCH_STATE\'].isin(["NJ", "CT", "MA"]),\n        F.concat(F.lit("0"), df_branch[\'BRANCH_ZIP\'])\n    ).otherwise(df_branch["BRANCH_ZIP"])\n)'

In [10]:
'''df_branch = df_branch.withColumn('BRANCH_PHONE',
            F.format_string("(%s)%s-%s", df_branch['BRANCH_PHONE'][0:3],
            df_branch['BRANCH_PHONE'][4:3],df_branch['BRANCH_PHONE'][7:4]))'''

'df_branch = df_branch.withColumn(\'BRANCH_PHONE\',\n            F.format_string("(%s)%s-%s", df_branch[\'BRANCH_PHONE\'][0:3],\n            df_branch[\'BRANCH_PHONE\'][4:3],df_branch[\'BRANCH_PHONE\'][7:4]))'

In [11]:
df_branch = df_branch.withColumn('BRANCH_PHONE',
            F.format_string("(%s)%s-%s", 
                            F.substring(df_branch['BRANCH_PHONE'], 1, 3), 
                            F.substring(df_branch['BRANCH_PHONE'], 4, 3), 
                            F.substring(df_branch['BRANCH_PHONE'], 7, 4)))


In [12]:
df_branch = df_branch.withColumn(
    'BRANCH_CITY',
    F.regexp_replace(df_branch['BRANCH_CITY'], "(?<=.)([A-Z])", ' $1')
)

In [13]:
# from pyspark.sql import functions as F

'''def format_zip(zip_code, state):
    if len(zip_code) == 4 and state in ["NJ", "CT", "MA"]:
        return "0" + zip_code
    return zip_code

# Convert the Python function to a PySpark UDF
format_zip_udf = F.udf(format_zip)

# Apply the UDF to the DataFrame
df_branch = df_branch.withColumn('BRANCH_ZIP', format_zip_udf(df_branch['BRANCH_ZIP'], df_branch['BRANCH_STATE']))
# df_branch.show()'''

'def format_zip(zip_code, state):\n    if len(zip_code) == 4 and state in ["NJ", "CT", "MA"]:\n        return "0" + zip_code\n    return zip_code\n\n# Convert the Python function to a PySpark UDF\nformat_zip_udf = F.udf(format_zip)\n\n# Apply the UDF to the DataFrame\ndf_branch = df_branch.withColumn(\'BRANCH_ZIP\', format_zip_udf(df_branch[\'BRANCH_ZIP\'], df_branch[\'BRANCH_STATE\']))\n# df_branch.show()'

In [14]:
df_branch.columns
df_branch = df_branch.select("BRANCH_CODE", "BRANCH_NAME", "BRANCH_STREET", "BRANCH_CITY", "BRANCH_STATE", "BRANCH_ZIP", "BRANCH_PHONE", "LAST_UPDATED")
df_branch.show()

+-----------+------------+-------------------+-------------------+------------+----------+-------------+--------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|        BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|        LAST_UPDATED|
+-----------+------------+-------------------+-------------------+------------+----------+-------------+--------------------+
|          1|Example Bank|       Bridle Court|          Lakeville|          MN|     55044|(123)456-5276|2018-04-18T16:51:...|
|          2|Example Bank|  Washington Street|            Huntley|          IL|     60142|(123)461-8993|2018-04-18T16:51:...|
|          3|Example Bank|      Warren Street|South Richmond Hill|          NY|     11419|(123)498-5926|2018-04-18T16:51:...|
|          4|Example Bank|   Cleveland Street|         Middleburg|          FL|     32068|(123)466-3064|2018-04-18T16:51:...|
|          5|Example Bank|        14th Street|    King Of Prussia|          PA|     19406|(123)484-9701|2018-04-18T16:

In [15]:
df_branch.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "creditcard_capstone.cdw_sapp_branch") \
  .option("user", "root") \
  .option("password", "password") \
  .save()

In [27]:
df_credit = spark.read.json("cdw_sapp_credit.json")
df_credit.show()

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|
|        160|4210653349028689|123459988|  8|    7|             3|         Grocery|             56.7|2018|
|        114|4210653349028689|123459988| 19|    4|             4|   Entertainment|            59.73|2018|
|         93|4210653349028689|123459988| 10|   10|             5|             Gas|             3.59|2018|
|        164|4210653349028689|123459988| 28|    5|             6|       Education|             6.89|2018|
|        119|4210653349028689|123459988| 19|  

In [28]:
df_credit.columns
df_credit.printSchema()
df_credit.describe().show()
df_credit.show()

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)

+-------+------------------+--------------------+-------------------+-----------------+------------------+------------------+----------------+------------------+--------------------+
|summary|       BRANCH_CODE|      CREDIT_CARD_NO|           CUST_SSN|              DAY|             MONTH|    TRANSACTION_ID|TRANSACTION_TYPE| TRANSACTION_VALUE|                YEAR|
+-------+------------------+--------------------+-------------------+-----------------+------------------+------------------+----------------+------------------+--------------------+
|  count|             46694|               46694|              46694|        

In [33]:
from pyspark.sql.functions import concat_ws

df_credit = df_credit.withColumn("TimeID", concat_ws('', df_credit['year'], 
    F.lpad(df_credit['month'], 2, '0'), 
    F.lpad(df_credit['day'], 2, '0')))

In [34]:
# create a temporary view of your DataFrame
df_credit.createOrReplaceTempView("tempTable")

# 
spark.sql("""
SELECT *, CONCAT(year, LPAD(month, 2, '0'), LPAD(day, 2, '0')) as TimeID
FROM tempTable
""")


DataFrame[BRANCH_CODE: bigint, CREDIT_CARD_NO: string, CUST_SSN: bigint, DAY: bigint, MONTH: bigint, TRANSACTION_ID: bigint, TRANSACTION_TYPE: string, TRANSACTION_VALUE: double, YEAR: bigint, TimeID: string, TimeID: string]

In [35]:
df_credit = df_credit.select('TRANSACTION_ID', 'BRANCH_CODE', 'CREDIT_CARD_NO', 'TRANSACTION_TYPE', 'TRANSACTION_VALUE', 'CUST_SSN', 'TIMEID')
df_credit.show()

+--------------+-----------+----------------+----------------+-----------------+---------+--------+
|TRANSACTION_ID|BRANCH_CODE|  CREDIT_CARD_NO|TRANSACTION_TYPE|TRANSACTION_VALUE| CUST_SSN|  TIMEID|
+--------------+-----------+----------------+----------------+-----------------+---------+--------+
|             1|        114|4210653349028689|       Education|             78.9|123459988|20180214|
|             2|         35|4210653349028689|   Entertainment|            14.24|123459988|20180320|
|             3|        160|4210653349028689|         Grocery|             56.7|123459988|20180708|
|             4|        114|4210653349028689|   Entertainment|            59.73|123459988|20180419|
|             5|         93|4210653349028689|             Gas|             3.59|123459988|20181010|
|             6|        164|4210653349028689|       Education|             6.89|123459988|20180528|
|             7|        119|4210653349028689|   Entertainment|            43.39|123459988|20180519|


In [ ]:
df_credit.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "creditcard_capstone.cdw_sapp_credit_card2") \
  .option("user", mysql_username) \
  .option("password", mysql_password) \
  .save()

In [ ]:
df_customer = spark.read.json("cdw_sapp_customer.json")
df_customer.show()

+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States| EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States| WDunham@exam

In [22]:
df_customer.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "creditcard_capstone.cdw_sapp_customer") \
  .option("user", mysql_username) \
  .option("password", mysql_password) \
  .save()

NameError: name 'df_customer' is not defined